# Hands-on Lab: Acquiring and Processing Information on the World's Largest Banks

## Project Scenario:

You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

| **Parameter**                             | **Value**                                                                                           |
|-------------------------------------------|-----------------------------------------------------------------------------------------------------|
| **Code name**                             | `banks_project.py`                                                                                  |
| **Data URL**                              | [https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks](https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks) |
| **Exchange rate CSV path**                | [https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv) |
| **Table Attributes (upon Extraction only)**| `Name`, `MC_USD_Billion`                                                                             |
| **Table Attributes (final)**              | `Name`, `MC_USD_Billion`, `MC_GBP_Billion`, `MC_EUR_Billion`, `MC_INR_Billion`                       |
| **Output CSV Path**                       | `./Largest_banks_data.csv`                                                                           |
| **Database name**                         | `Banks.db`                                                                                           |
| **Table name**                            | `Largest_banks`                                                                                      |
| **Log file**                              | `code_log.txt`                                                                                       |


## Requirements

In [1]:
!pip install numpy pandas bs4 wget > /dev/null

In [2]:
import numpy as np
import pandas as pd
import requests
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime
import wget
# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [3]:
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")

'exchange_rate (1).csv'

## Project tasks

### Task 1:

Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

In [4]:
def log_progress(message):
    timestamp_format = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + " : " + message + "\n")

### Task 2:

Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

In [18]:
def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")

    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all("tbody")
    rows = tables[0].find_all("tr")

    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df

### Task 3:

Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

In [6]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv(csv_path)

    exchange_rate = exchange_rate.set_index("Currency").to_dict()["Rate"]

    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]

    return df

### Task 4:

Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

In [7]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)


### Task 5:

Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

In [8]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


### Task 6:

Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

In [9]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [10]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "./exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
output_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file = "./code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process")

In [22]:
df = extract(url, table_attribs)
log_progress("Data extraction complete. Initiating Transformation process")
df

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [23]:
df = transform(df, csv_path)
log_progress("Data transformation complete. Initiating Loading process")
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [ ]:
load_to_csv(df, output_path)
log_progress("Data saved to CSV file")

In [ ]:
sql_connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

In [11]:
load_to_db(df, sql_connection, table_name)
log_progress("Data loaded to Database as a table, Executing queries")

In [12]:
query_statement = f"SELECT * from Largest_banks"
run_query(query_statement, sql_connection)

SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

In [13]:
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


In [14]:
query_statement = f"SELECT Name from Largest_banks LIMIT 15"
run_query(query_statement, sql_connection)

SELECT Name from Largest_banks LIMIT 15
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank
5                              Wells Fargo
6                                     HSBC
7                           Morgan Stanley
8                  China Construction Bank
9                            Bank of China


In [15]:
sql_connection.close()

### Task 7:

Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [16]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

2024-Oct-07-23:07:08 : Preliminaries complete. Initiating ETL process
2024-Oct-07-23:14:09 : Data loaded to Database as a table, Executing queries
2024-Oct-07-23:16:07 : Preliminaries complete. Initiating ETL process
2024-Oct-07-23:16:07 : Data extraction complete. Initiating Transformation process
2024-Oct-07-23:16:07 : Data transformation complete. Initiating Loading process
2024-Oct-07-23:16:07 : Data saved to CSV file
2024-Oct-07-23:16:07 : SQL Connection initiated
2024-Oct-07-23:16:07 : Data loaded to Database as a table, Executing queries

